# Random Forest - Lending Club
<br>
Predict the state of a loan given some information about it. 
<br>
<br> link to dataset:
https://www.lendingclub.com/info/download-data.action
<br>
<br>
We'll use 2015 data.
<br>
<br>
Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

In [70]:
# Import modules.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
import time

In [94]:
# Load dataset.

y2015 = pd.read_csv('~/src/data/unit3/LoanStats3d.csv', skipinitialspace=True, header=1)
y2015.head()

c:\users\mewing_user\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


## Data Cleaning

In [84]:
y2015.dtypes

id                                 object
member_id                         float64
loan_amnt                         float64
funded_amnt                       float64
funded_amnt_inv                   float64
term                               object
int_rate                           object
installment                       float64
grade                              object
sub_grade                          object
emp_title                          object
emp_length                         object
home_ownership                     object
annual_inc                        float64
verification_status                object
issue_d                            object
loan_status                        object
pymnt_plan                         object
url                                object
desc                               object
purpose                            object
title                              object
zip_code                           object
addr_state                        

In [95]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


Some of these have > 100k distinct types. Let's drop the ones with > 30 unique values, converting to numeric where it makes sense.

In [96]:
# Convert id and int_rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables.
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [97]:
y2015.isnull().sum()

id                                     2
member_id                              2
loan_amnt                              2
funded_amnt                            2
funded_amnt_inv                        2
term                                   2
int_rate                               2
installment                            2
grade                                  2
emp_length                         23819
home_ownership                         2
annual_inc                             2
verification_status                    2
issue_d                                2
loan_status                            2
pymnt_plan                             2
purpose                                2
title                                134
dti                                    2
delinq_2yrs                            2
inq_last_6mths                         2
mths_since_last_delinq            203964
mths_since_last_record            346682
open_acc                               2
pub_rec         

In [98]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
# Remove two summary rows at the end that don't actually contain data.
y2015_df1 = y2015[:-2]

In [105]:
y2015_df1.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421090,36371250.0,39102635.0,10000.0,10000.0,10000.0,36 months,11.99,332.10,B,8 years,...,0.0,1.0,100.0,100.0,0.0,0.0,32950.0,25274.0,9200.0,15850.0
421091,36441262.0,39152692.0,24000.0,24000.0,24000.0,36 months,11.99,797.03,B,10+ years,...,0.0,2.0,56.5,100.0,0.0,0.0,152650.0,8621.0,9000.0,0.0
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0


## Run the model

In [91]:
rfc = ensemble.RandomForestClassifier()
X = y2015_df1.drop('loan_status', 1)
Y = y2015_df1['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)
cross_val_score(rfc, X, Y, cv=10)

array([0.97846168, 0.98002897, 0.98183373, 0.98176248, 0.97233436,
       0.97508905, 0.94217388, 0.98071671, 0.97926711, 0.98019285])

#### Analysis:
Here, we're about 98% accurate which works well but presents a few potential problems. We did not do much in the way of feature selection or model refinement. As such, there are a lot of features we don't really need. There's also some variance in the scores (94% accuracy vs 98%). This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## Data Cleaning (pt 2)

In [106]:
# Removing id & member_id.
y2015_df1.drop(['id', 'member_id'], axis=1, inplace=True)

c:\users\mewing_user\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [107]:
categorical = y2015_df1.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

term
2
grade
7
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
purpose
14
title
27
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [108]:
y2015_df1.drop(['grade', 'issue_d', 'verification_status', 'title'], axis=1, inplace=True)

c:\users\mewing_user\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [109]:
y2015_dummies = pd.get_dummies(y2015_df1)

In [110]:
# Build correlation matrix.
corrmat = y2015_dummies.corr()
print(corrmat.shape)
corrmat.head()

(186, 186)


,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
loan_amnt,1.000000,1.000000,0.999994,0.140572,0.941205,0.305734,0.006103,-0.010424,-0.039040,-0.034863,...,0.003849,-0.000398,0.009984,0.000568,-0.004610,-0.016880,0.016880,0.008359,0.007089,0.014350
funded_amnt,1.000000,1.000000,0.999994,0.140572,0.941205,0.305734,0.006103,-0.010424,-0.039040,-0.034863,...,0.003849,-0.000398,0.009984,0.000568,-0.004610,-0.016880,0.016880,0.008359,0.007089,0.014350
funded_amnt_inv,0.999994,0.999994,1.000000,0.140209,0.941187,0.305803,0.006044,-0.010448,-0.039103,-0.034833,...,0.003850,-0.000394,0.009988,0.000578,-0.004605,-0.016832,0.016832,0.008338,0.007071,0.014305
int_rate,0.140572,0.140572,0.140209,1.000000,0.124426,-0.090399,0.077932,0.043710,0.231139,-0.015459,...,0.012297,0.007406,0.078885,0.004256,0.013182,-0.020755,0.020755,0.008172,0.011635,0.018622
installment,0.941205,0.941205,0.941187,0.124426,1.000000,0.297393,0.001893,-0.002129,-0.006255,-0.037046,...,0.006806,0.001181,0.018691,0.002285,-0.002827,-0.015129,0.015129,0.006155,0.006336,0.014613


In [111]:
print(corrmat.columns[0:51])
print(corrmat.columns[51:100])
print(corrmat.columns[101:150])
print(corrmat.columns[151:200])
print(corrmat.columns[201:])

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate',
       'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'total_acc', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'annual_inc_joint',
       'dti_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal',
       'bc_open_to_buy'],
      dtype='object')
Index(['bc_util', 'chargeoff

In [112]:
# Remove variables that are unrelated to loan_status
loan_status = ['loan_status_Fully Paid', 'loan_status_Current', 'loan_status_Charged Off',
               'loan_status_Late (31-120 days)', 'loan_status_In Grace Period',
               'loan_status_Default', 'loan_status_Late (16-30 days)']

# Construct corr matrix for loan status.
corrmat_loan_status = corrmat[loan_status]

# Create new column summing each row.
corrmat_loan_status['summed'] = [sum(corrmat_loan_status.iloc[j])
                                 for j in range(0, len(corrmat_loan_status))]

# Sort 'summed' column.
corrmat_loan_status['summed'].sort_values(ascending=False)

c:\users\mewing_user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


loan_status_Default               0.887982
loan_status_Late (16-30 days)     0.828841
loan_status_In Grace Period       0.743873
loan_status_Late (31-120 days)    0.625224
loan_status_Charged Off           0.360243
last_pymnt_d_Aug-2016             0.265382
last_pymnt_d_Dec-2016             0.228519
int_rate                          0.227989
last_pymnt_d_Sep-2016             0.217299
last_pymnt_d_Nov-2016             0.182027
total_rec_late_fee                0.176385
last_pymnt_d_Oct-2016             0.175733
collection_recovery_fee           0.167451
recoveries                        0.166772
last_credit_pull_d_Oct-2016       0.122148
acc_open_past_24mths              0.119297
num_tl_op_past_12m                0.107263
open_il_24m                       0.103631
open_il_12m                       0.102597
inq_last_12m                      0.098213
open_acc_6m                       0.095215
inq_last_6mths                    0.085546
open_rv_24m                       0.080618
open_rv_12m

In [119]:
# Select variables both pos & neg relating to balance & payment.
final_vars = ['last_pymnt_d_Aug-2016', 'last_pymnt_d_Dec-2016', 'last_pymnt_d_Sep-2016', 'last_pymnt_d_Nov-2016',
              'last_pymnt_d_Oct-2016', 'last_pymnt_d_Jun-2016', 'tot_cur_bal', 'avg_cur_bal', 'last_pymnt_amnt',
              'total_pymnt', 'total_pymnt_inv', 'next_pymnt_d_Feb-2017', 'total_rec_prncp', 'last_pymnt_d_Jan-2017']

In [120]:
vars_X = X[final_vars]

In [121]:
cross_val_score(rfc, vars_X, Y, cv=10)

array([0.97352236, 0.9750184 , 0.9768469 , 0.97808174, 0.97527903,
       0.97729755, 0.97708328, 0.97665582, 0.97489729, 0.97606042])

Better, we can tune more.